In [28]:
from articles import yield_wiki_articles
from parse_wiki_markup import parse_wiki_markup

In [29]:
SMALL_FILE = r"C:\\Users\\Lenovo\\Downloads\\wikidump\\enwiki-20250501-pages-articles-multistream-index.txt.bz2"
BIG_FILE = r"C:\\Users\\Lenovo\\Downloads\\wikidump\\enwiki-20250501-pages-articles.xml.bz2"

articles = yield_wiki_articles(BIG_FILE)
    


In [30]:
article = next(articles)

In [34]:
found = 0
not_found = 0
while found < 10:
    article = next(articles)
    if "birthplace" in article['text'].lower():
        found += 1
    not_found +=1 
print(found, not_found)

10 170


In [32]:
article['text']

'{{Short description|Country in Southeast Europe}}\n{{About|the country in Southeastern Europe|other uses|Albania (disambiguation)}}\n{{Pp-extended|small=yes}}\n{{Use dmy dates|date=August 2024}}\n{{Use British English|date=January 2022}}\n{{Infobox country\n| conventional_long_name = Republic of Albania\n| native_name            = {{native name|sq|Republika e Shqipërisë}}\n| common_name            = Albania\n| image_flag             = Flag of Albania.svg\n| alt_flag               = Red flag with a black double-headed eagle in the centre.\n| image_coat             = [[File:Coat of arms of Albania.svg|60px]]\n| national_motto         = {{native phrase|sq|[[Ti Shqipëri, më jep nder, më jep emrin Shqipëtar|Ti Shqipëri, më jep nder,<br>më jep emrin Shqipëtar]]|paren=no|italic=yes}}<br>"You Albania, you give me honour,<br>you give me the name Albanian"\n| national_anthem        = {{lang|sq|"[[Himni i Flamurit]]"|italic=no}}<br>"Hymn to the Flag"<br/><div style="display:inline-block;margin-t

In [33]:
parse_wiki_markup(article['text'])

{'tables': ['{| class="wikitable" style="text-align:left; float:right; margin-right:9px; margin-left:2px;"\n|-\n| style="text-align:left;" | [[File:Bajram Begaj (infobox crop).jpg|108px]]\n| style="text-align:left;" | [[File:Edi Rama (2024-02-29).jpg|120px]]\n|-\n| style="text-align:center;" | [[Bajram Begaj]]<br/>{{Small|[[President of Albania|President]]}}\n| style="text-align:center;" | [[Edi Rama]]<br/>{{Small|[[Prime Minister of Albania|Prime Minister]]}}\n|}',
  '{| class="sortable wikitable" style="text-align:left; font-size:85%; margin:1em auto;"\n|- style="font-size:100%; text-align:right;"\n! style="width:25px;" | [[Coat of arms|Emblem]] !! style="width:100px;"| [[Counties of Albania|County]] !! style="width:87px;"| Capital !! style="width:75px;" | Area <br/>(km<sup>2</sup>) !! style="width:70px;"| [[List of counties of Albania by population|Population]] (2020) !! style="width:70px;" | [[List of counties of Albania by Human Development Index|HDI]] (2019)\n|-\n! scope="row" st